In [2]:
from data.Algorithem.global_repository import game_status_repository,player_status_repository,round_status_repository,add_player,del_player
from data.Algorithem.gameready import card_distribution,round_card_shuffle,start_resource_distribution

game_status_repository.now_turn_player=1
import socket,pickle
class Server_Client():
    def __init__(self,name="HOST"):
        global game_status_repository,round_status_repository,player_status_repository
        game_status_repository.__init__()
        player_status_repository = {}
        #포트 조회
        self.name = name
        # self.server_socket = self.start_server()
        self.player = {}
        # self.start_recieved()
        self.maximum_count = 4
        pass

    def start_server(self,host='0.0.0.0', port=12345):
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((host, port))
        server_socket.listen(5)
        print(f'Server listening on {host}:{port}') 
        self.status = "greenroom"
        return server_socket

    def start_recived(self):
        global game_status_repository,round_status_repository,player_status_repository
        game_status_repository.status = "greenroom"
        self.server_socket = self.start_server()
        while True:
            
            client_socket, addr = self.server_socket.accept()
            client_ip = addr[0]
            print(f'Connection from {client_ip}')
            data_ori = pickle.loads(client_socket.recv(9192))
            print(data_ori)


            if len(data_ori)<5:
                player = data_ori[1]
                data = data_ori[0]
                match data:
                    case "ping":
                        client_socket.send(pickle.dumps(['pong',self.name,len(self.player)]))
                    case "OPEN?":
                        client_socket.send(pickle.dumps(['yes',]))
                    case "JOIN":
                        if len(self.player) ==self.maximum_count and (client_ip,player) not in self.player :
                            client_socket.send(pickle.dumps(['Full',]))
                        else:
                            print(f"{player} 플레이어가 입장했습니다. ")
                            player_status_repository = add_player(player,client_ip)
                            game_status_repository.num_players+=1
                            self.player[client_ip,player]=client_socket
                            client_socket.send(pickle.dumps([game_status_repository,round_status_repository,player_status_repository,"",""]))
                            print(self.player.keys())




                    case "exit":
                        del self.player[client_ip]
                        del_player(player)
                        game_status_repository.num_players-=1
                        client_socket.send(pickle.dumps(f'yes'))
                        print(self.player)
                        print(f"{player} 플레이어가 나갔습니다.")




                    case _:
                        client_socket.send(pickle.dumps(f'error'))
                        print("error")
            elif len(data_ori)>=4:
                player = data_ori[-1]
                data = data_ori[:-1]
                self.player[client_ip,player]=client_socket
                player_status_repository[player] = data[2][player]
                game_status_repository = data[0]
                round_status_repository = data[1]
                print(game_status_repository.status)
                if (False not in [player_status_repository[player].ready for player in player_status_repository.keys()]) and len(player_status_repository)>1 and game_status_repository.status == "greenroom":
                    print( [player_status_repository[player].ready for player in player_status_repository.keys()],self.status)
                    print("게임이 시작되었습니다.")
                    game_status_repository.status = "start"
                    game_status_repository.players = list(player_status_repository.keys())
                    game_status_repository.now_turn_player = list(player_status_repository.keys())[0]
                    card_distribution.CardDistribution()
                    round_card_shuffle.RoundCardShuffle()
                    start_resource_distribution.StartResourceDistribution()

                    game_status_repository.next = True
                    print(game_status_repository.now_turn_player)
                print("game_status_repository.next:",game_status_repository.next)
                if game_status_repository.next:
                    game_status_repository.next = False
                    for key,client in self.player.items():
                        print(f"데이터를 전송했습니다 : {key}")
                        client.send(pickle.dumps([game_status_repository,round_status_repository,player_status_repository,"",""]))
        
        
        
        
        


In [3]:
try:
    a.server_socket.close()
except:pass
a = Server_Client()

a.start_recived()

Server listening on 0.0.0.0:12345
Connection from 192.168.0.5
['ping', '김도훈d']
Connection from 192.168.0.5
['ping', '김도훈']
Connection from 192.168.0.5
['JOIN', '김도훈d', '김도훈d']
김도훈d 플레이어가 입장했습니다. 
dict_keys([('192.168.0.5', '김도훈d')])
Connection from 192.168.0.5
['JOIN', '김도훈', '김도훈']
김도훈 플레이어가 입장했습니다. 
dict_keys([('192.168.0.5', '김도훈d'), ('192.168.0.5', '김도훈')])
Connection from 192.168.0.5
[<data.Algorithem.repository.game_status.GameStatus object at 0x000001C4C8246AE0>, <data.Algorithem.repository.round_status.RoundStatus object at 0x000001C4C825F0B0>, {'김도훈d': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000001C4C825F140>, '김도훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x000001C4C825D910>}, '', '', '김도훈', '김도훈']
greenroom
game_status_repository.next: False
Connection from 192.168.0.5
[<data.Algorithem.repository.game_status.GameStatus object at 0x000001C4C8246BA0>, <data.Algorithem.repository.round_status.RoundStatus object at 0x000001C4C

Server listening on 0.0.0.0:12345
Connection from 192.168.0.5
['JOIN', '김성훈', '김성훈']
김성훈 플레이어가 입장했습니다. 
dict_keys([('192.168.0.5', '김성훈')])
Connection from 192.168.0.5
['JOIN', '김성훈', '김성훈']
김성훈 플레이어가 입장했습니다. 
dict_keys([('192.168.0.5', '김성훈')])
Connection from 192.168.0.5
[<data.Algorithem.repository.game_status.GameStatus object at 0x0000028F9D006510>, <data.Algorithem.repository.round_status.RoundStatus object at 0x0000028F9D02CC20>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x0000028F9D02E6F0>}, '김성훈', '김성훈']
greenroom
Connection from 192.168.0.5
['JOIN', '김도훈', '김도훈']
김도훈 플레이어가 입장했습니다. 
dict_keys([('192.168.0.5', '김성훈'), ('192.168.0.5', '김도훈')])
Connection from 192.168.0.5
[<data.Algorithem.repository.game_status.GameStatus object at 0x0000028F9BD7BEC0>, <data.Algorithem.repository.round_status.RoundStatus object at 0x0000028F9D066660>, {'김성훈': <data.Algorithem.repository.player_status.PlayerStatus object at 0x0000028F9D064650>, '김도훈': <data.Algorith

In [7]:
game_status_repository.status

'greenroom'

In [4]:
a.player

{('192.168.0.5',
  '김도훈'): [<socket.socket fd=1916, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 56594)>, 0],
 ('192.168.0.5',
  '김선후'): [<socket.socket fd=404, family=2, type=1, proto=0, laddr=('192.168.0.5', 12345), raddr=('192.168.0.5', 56596)>, 1]}

In [7]:
a.player.keys()

dict_keys([('192.168.0.5', '김도훈'), ('192.168.0.5', '김선후')])

In [9]:
for l in a.player:
    print(l)

('192.168.0.5', '김도훈')
('192.168.0.5', '김선후')


In [ ]:
from data.Algorithem.global_repository import*
from data.Algorithem.gameready import card_distribution,round_card_shuffle,start_resource_distribution
card_distribution.CardDistribution()
round_card_shuffle.RoundCardShuffle()
start_resource_distribution.StartResourceDistribution()
game_status_repository.now_turn_player=1
import socket,pickle
class Server_Client():
    def __init__(self,name="HOST"):
        #포트 조회
        self.name = name
        # self.server_socket = self.start_server()
        self.player = {}
        self.play_state = "greenroom"
        # self.start_recieved()
        self.maximum_count = 2
        pass

    def start_server(self,host='0.0.0.0', port=12345):
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((host, port))
        server_socket.listen(5)
        print(f'Server listening on {host}:{port}') 
        return server_socket

    def start_recived(self):
        global game_status_repository,round_status_repository,player_status_repository
        game_status_repository.status = "ready"
        self.server_socket = self.start_server()
        while len(self.player) <self.maximum_count and self.play_state == "greenroom":
            client_socket, addr = self.server_socket.accept()
            client_ip = addr[0]
            print(f'Connection from {client_ip}')
            data_ori = client_socket.recv(1024).decode()
            print(data_ori)
            if "|" in data_ori:
                data = data_ori.split("|")[0]
                player = data_ori.split("|")[1]
            else:data=""
            match data:
                case "ping":
                    client_socket.send(f'pong|{self.name}|{len(self.player)}'.encode())
                case "OPEN?":
                    client_socket.send(f'yes'.encode())
                case "JOIN":
                    if len(self.player) ==self.maximum_count :
                        client_socket.send(f'Full'.encode())
                    else:
                        # client_socket.send(f'yes'.encode())
                        print(f"{player} 플레이어가 입장했습니다.")
                        self.player[client_ip,player]=[client_socket,len(self.player)]
                        print(self.player)
                # case "start":
                #     client_socket.send(f'Full'.encode())
                case "exit":
                    del self.player[client_ip]
                    client_socket.send(f'yes'.encode())
                    print(self.player)
                    print(f"{player} 플레이어가 나갔습니다.")
                case _:
                    client_socket.send(f'error'.encode())
                    print("error")
        print("게임이 시작되었습니다.")
        self.play_state="playing"
        # while self.play_state == "playing":
        for i in range(len(self.player)):
            player_status_repository[i].name=list(self.player.keys())[i]
        
        for client_socket,player_num in self.player.values():  
            game_status_repository.status = "start"       
            client_socket.send(pickle.dumps([game_status_repository,round_status_repository,player_status_repository]))

        while self.play_state == "playing":
            client_socket, addr = self.server_socket.accept()
            client_ip = addr[0]
            data_ori = client_socket.recv(9192)
            # print(f'Connection from {client_ip}')
            for i in self.player:
                if client_ip in i:
                    self.player[i][0]= client_socket
            if "wait|".encode() in data_ori :
                data = data_ori.decode().split("|")[0]
                player = data_ori.decode().split("|")[1]
                # self.player[client_ip,player][0] = client_socket
            elif "exit|".encode() in data_ori :
                client_socket.send(f'yes'.encode())
            else:
                print(data_ori)
                response = pickle.loads(data_ori)
                game_status_repository = response[0]
                round_status_repository = response[1]
                player_status_repository = response[2]
                for client,player_num in self.player.values():
                    # client_socket.send(data_ori)
                    client.send(pickle.dumps([game_status_repository,round_status_repository,player_status_repository]))
            # client_socket.send(data_ori)
            # client_socket.send(pick|e.dumps([game_status_repository,round_status_repository,player_status_repository]))
            if "exit".encode() in data_ori:
                self.play_state = ""
            print(game_status_repository.now_turn_player)
        self.server_socket.close()
        